In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
from idaes.core.util.misc import extract_data
import pyomo.environ as pyo
from pyomo.util.calc_var_value import calculate_variable_from_constraint
from State_Block import Create_State_Block
from idaes.core.util.model_statistics import degrees_of_freedom
from Unit_Model import F101_creater

In [2]:
# Create a concrete model
m = ConcreteModel()
# Define a set for components
m.Phase = Set(initialize = ['Liq','Vap'])
m.Comp = Set(initialize = ['benzene','toluene','methane','hydrogen'])

In [3]:
m.F101 = F101_creater(m)

In [4]:
# 各组分进料
m.F101.inlet.flow['Liq','benzene'].fix(1e-8)
m.F101.inlet.flow['Liq','toluene'].fix(1e-8)
m.F101.inlet.flow['Liq','methane'].fix(1e-8)
m.F101.inlet.flow['Liq','hydrogen'].fix(1e-8)
m.F101.inlet.flow['Vap','benzene'].fix(0.35374)
m.F101.inlet.flow['Vap','toluene'].fix(0.078129)
m.F101.inlet.flow['Vap','methane'].fix(1.2721)
m.F101.inlet.flow['Vap','hydrogen'].fix(0.32821)

# 温度压力
m.F101.inlet.P.fix(350000)
m.F101.inlet.T.fix(771.85)

m.F101.outlet.P.fix(350000)
m.F101.outlet.T.fix(325)

In [5]:
def Initialize(b):
    #calculate_variable_from_constraint(b.temperature_bubble, b.eq_T_bubble)
    #b.temperature_bubble.fix()
    #b.del_component(b.eq_T_bubble)
    b.temperature_bubble.fix(50)
    
    calculate_variable_from_constraint(b.temperature_dew, b.eq_T_dew)
    b.temperature_dew.fix()
    b.del_component(b.eq_T_dew)

    calculate_variable_from_constraint(b._t1, b._t1_constraint)
    b._t1.fix()
    b.del_component(b._t1_constraint)

    calculate_variable_from_constraint(b._teq, b._teq_constraint)
    b._teq.fix()
    b.del_component(b._teq_constraint)

Initialize(m.F101)

In [6]:
# 模型求解
solver = pyo.SolverFactory('ipopt')
results = solver.solve(m,tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [7]:
m.F101.pprint()

F101 : Size=1, Index=None, Active=True
    7 Set Declarations
        Comp : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'benzene', 'toluene', 'methane', 'hydrogen'}
        Phase : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Liq', 'Vap'}
        cp_ig_1_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : F101.Phase*F101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), ('Liq', 'hydrogen'), ('Vap', 'benzene'), ('Vap', 'toluene'), ('Vap', 'methane'), ('Vap', 'hydrogen')}
        cp_ig_2_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : F101.Phase*F101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), 

In [8]:
print(degrees_of_freedom(m))

0
